In [1]:
from clustering_utils import *

In [2]:
import pickle
tid_molecules_list =pickle.load(open('tid_molecules_list.p','rb'))

In [3]:
single_uncovered = pickle.load(open('../1.add-additional-substituents/single_uncovered.p','rb'))

In [4]:
# select core, choose around 10 entries per torsion parameter
# resulting tid_clusters_list[tid] = [ {'cluster_label': N, 'torsions': [...]}]
tid_clusters_list = gen_tid_clusters_list(tid_molecules_list, fptype=OEFPType_MACCS166, lim=10, select_option =1) 

tid: t1
# eps: 0.40, Ncluster: 3
# eps: 0.38, Ncluster: 9
tot: 99, eps: 0.38, Ncluster: 9
tid: t2
tid: t3
# eps: 0.40, Ncluster: 3
# eps: 0.38, Ncluster: 9
tot: 101, eps: 0.38, Ncluster: 9
tid: t4
# eps: 0.40, Ncluster: 4
# eps: 0.38, Ncluster: 8
tot: 75, eps: 0.38, Ncluster: 8
tid: t5
tid: t6
tid: t7
tid: t8
tot: 1, Ncluster: 1
tid: t9
tot: 10, Ncluster: 10
tid: t10
tot: 3, Ncluster: 3
tid: t11
tot: 1, Ncluster: 1
tid: t12
# eps: 0.40, Ncluster: 3
# eps: 0.38, Ncluster: 9
tot: 101, eps: 0.38, Ncluster: 9
tid: t13
tid: t14
tot: 1, Ncluster: 1
tid: t15
tid: t16
tid: t17
tid: t20
tid: t18
tid: t19
tid: t21
tid: t22
tid: t23
tid: t24
tid: t25
tid: t26
tid: t27
# eps: 0.40, Ncluster: 6
# eps: 0.38, Ncluster: 8
tot: 90, eps: 0.38, Ncluster: 8
tid: t28
tot: 5, Ncluster: 5
tid: t29
tot: 5, Ncluster: 5
tid: t30
tot: 8, Ncluster: 8
tid: t31
# eps: 0.40, Ncluster: 2
# eps: 0.38, Ncluster: 2
# eps: 0.36, Ncluster: 3
# eps: 0.34, Ncluster: 4
# eps: 0.32, Ncluster: 4
# eps: 0.30, Ncluster: 4
# eps:

In [5]:
tid_clusters_list_interest = dict()
for tid in tid_clusters_list.keys():
    if tid in single_uncovered: 
        tid_clusters_list_interest[tid] = tid_clusters_list[tid]

In [6]:
print(f'avg: {sum([len(clusters) for tid, clusters in tid_clusters_list_interest.items() if len(clusters)>0])/sum([1 for tid, clusters in tid_clusters_list_interest.items() if len(clusters)>0]):.2f}')
print(f'max: {max([len(clusters) for tid, clusters in tid_clusters_list_interest.items()])}')
print(f'min: {min([len(clusters) for tid, clusters in tid_clusters_list_interest.items() if len(clusters)>0])}')

avg: 9.11
max: 13
min: 1


In [7]:
count = 0
tot = 0
for tid, clusters in tid_clusters_list_interest.items():
    if len(clusters) == 0:
        print(tid)
        count += 1
    tot += len(clusters)
print(f'# uncovered: {count} (out of {len(tid_clusters_list_interest)})')
print(f'# entries: {tot}')
# t68 center bond is endocyclic single bond and t157 has triple bond in one end:)

t68
t157
# uncovered: 2 (out of 20)
# entries: 164


In [8]:
import pickle
pickle.dump(tid_clusters_list_interest, open('tid_clusters_list_interest.p','wb'))

In [9]:
from tqdm import tqdm
from openff.toolkit.topology import Molecule
from openff.toolkit.utils import UndefinedStereochemistryError

to_skip = []
for tid, clusters in tqdm(tid_clusters_list_interest.items()):
    for cluster in clusters:
        torsions = cluster['torsions']
        for torsion in torsions:
            smiles_pattern = torsion['mol_index']
            try:
                molecule = Molecule.from_smiles(smiles_pattern)
            except UndefinedStereochemistryError:
                molecule = Molecule.from_smiles(smiles_pattern, allow_undefined_stereo=True)
            try:
                molecule = molecule.canonical_order_atoms()
                molecule.generate_conformers(n_conformers=10)
                molecule.to_rdkit()
            except:
                to_skip.append(smiles_pattern)
print(to_skip)

 85%|████████▌ | 17/20 [00:24<00:04,  1.55s/it]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 16, name: , idx: 4, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 8, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 12, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 2, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 7, aromatic: False, chiral: False
bond order: 1, chiral: 